In [1]:
import pandas as pd
import numpy as np
path = ''
import gc
import xgboost as xgb
#from sklearn.cross_validation import train_test_split 
from sklearn.metrics import mean_squared_error as rmse
from sklearn.cross_validation import KFold
import sys
#sys.path.append(path+'/src/python/')
from data_load import loadHDF_train
from data_load import loadHDF_test
from time import time
from sys import stdout

from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn import metrics

outputfilehead = 'random_forests_1hrAgg'
finalsubmitfile = '0912_'+outputfilehead+'.csv'

colnames = [u'timestamp', u'time_hr', u'meanInv.custid', u'q75Inv.custid', u'q95Inv.custid',
            u'q999Inv.custid', u'rInvGT10.custid', u'rInvGT2.custid',
            u'meanInv.curHr', u'q75Inv.curHr', u'q95Inv.curHr', u'q999Inv.curHr',
            u'rInvGT10.curHr', u'rInvGT2.curHr', u'meanInv.pltfHr',
            u'q75Inv.pltfHr', u'q95Inv.pltfHr', u'q999Inv.pltfHr',
            u'rInvGT10.pltfHr', u'rInvGT2.pltfHr', u'meanInv.grpidHr',
            u'q75Inv.grpidHr', u'q95Inv.grpidHr', u'q999Inv.grpidHr',
            u'rInvGT10.grpidHr', u'rInvGT2.grpidHr']
datafilehead = path + 'feat_1hr_'
timelimit = 3600.*24*np.array([153,183]) # last 30 days 

def preprocess(d, timelimit=[0,999999], missing=0, index = []):
    if(len(index) > 0):
        index = index[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])]
        d = d[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])]

    d.drop('timestamp', inplace=True, axis=1)
    d.fillna(missing, inplace=True)
    if(len(index) == 0):
        return d
    else:
        return d, index

dtr, pred, dtr_index = loadHDF_train(datafilehead+'train.h5', key='train', 
                                     selection = 'index > {0}'.format(35730865),\
                                     columns = colnames)

## preprocess
dtr, dtr_index = preprocess(dtr, timelimit = timelimit, missing=0, index=dtr_index)
pred = preprocess(pred, timelimit, missing=0)

dte, dte_index = loadHDF_test(datafilehead+'test.h5', key='test', columns = colnames)
dte = preprocess(dte, missing=0)
num_te = dte.shape[0]

X_train = dtr
y_train = pred
y_train = np.ravel(y_train)
X_test = dte

# Random forests
forest_model = RandomForestRegressor(n_estimators=300,\
                                     max_depth = 5,\
                                     min_samples_split=2,\
                                     min_samples_leaf=1,\
                                     max_features='log2')
forest_model.fit(X_train, y_train)
y_pred_forest = forest_model.predict(X_test)

In [2]:
print len(y_pred_forest)
print y_pred_forest
np.savetxt(path+finalsubmitfile, y_pred_forest, fmt='%.14e')
# y_pred_forest.to_csv(path+finalsubmitfile,header=False,index=False)

4606743
[ 1.13228283  1.13228283  1.13228283 ...,  1.19612139  1.19612139
  1.19612139]
